In [ ]:
#######################   Indicadores SAT   ########################
import os
import pandas as pd
import xlsxwriter
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, PatternFill, Border, Side, numbers

Fecha = "04_Abril 2025"
Base = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Proyecto INOVA\Colab_Notebooks/"
ruta_R4_Sub = f"{Base}/Indicadores/Procesos BDUA/Subsidiado/R4"
ruta_S4_Sub = f"{Base}/Indicadores/Procesos BDUA/Subsidiado/S4"
ruta_S1_Auto = f"{Base}/Indicadores/Procesos BDUA/Subsidiado/S1_AUTO"
ruta_S1_Val = f"{Base}/Indicadores/Procesos BDUA/Subsidiado/S1_VAL"
ruta_S3 = f"{Base}/Indicadores/Procesos BDUA/Subsidiado/S3"
ruta_S5 = f"{Base}/Indicadores/Procesos BDUA/Subsidiado/S5"
ruta_S6 = f"{Base}/Indicadores/Procesos BDUA/Subsidiado/S6"
ruta_R4_Cont = f"{Base}/Indicadores/Procesos BDUA/Contributivo/R4"
ruta_S4_Cont = f"{Base}/Indicadores/Procesos BDUA/Contributivo/S4"
# Especifica la ruta y el nombre del archivo Excel de salida
ruta_excel_salida = f"{Base}/Indicadores/Estadisticas Traslado BDUA {Fecha}_CORRECCION.xlsx"  # Cambia esto a la ruta que desees

########## Función #1
def cargar_archivos_a_dataframe(ruta_carpeta, tipos_archivo, codificacion_archivo, separador, tiene_encabezado):
    dfs = []  # Cambia el nombre de la lista a "dfs"

    # Itera sobre los archivos en la carpeta
    for filename in os.listdir(ruta_carpeta):
        # Verifica si el archivo coincide con los tipos especificados
        if filename.endswith(tuple(tipos_archivo)):
            filepath = os.path.join(ruta_carpeta, filename)
            with open(filepath, 'r', encoding=codificacion_archivo) as file:  
                lines = file.readlines()
                # Elimina la última línea vacía (si existe)
                if lines and lines[-1] == '':
                    lines.pop()
                # Verifica si las líneas están vacías o contienen solo espacios
                lines = [line for line in lines if line.strip() != '']
                # Si tiene encabezado y no es el primer archivo, se omite el encabezado
                if tiene_encabezado and len(dfs) > 0:
                    lines = lines[1:]
                # Agrega el nombre del archivo como una columna adicional
                for i in range(len(lines)):
                    lines[i] = filename + separador + lines[i]
                # Crea un DataFrame a partir de las líneas modificadas
                df = pd.DataFrame(lines, columns=["Registro"], dtype=str)
                # Agrega el DataFrame a la lista "dfs"
                dfs.append(df)


    # Concatena todos los DataFrames en la lista
    df_retorno = pd.concat(dfs, ignore_index=True)
    # Divide la columna "Registro" en múltiples columnas usando el separador especificado
    df_retorno = df_retorno["Registro"].str.split(separador, expand=True)
    # Asigna nombres de columnas a las nuevas columnas
    df_retorno.columns = ["Nombre_Archivo"] + [f"Columna_{i}" for i in range(1, df_retorno.shape[1])]

    return df_retorno


Tipo_Archivo = '.TXT'
Codificacion_Archivo = 'ANSI'
Separador_Archivo = ','
tiene_encabezado = False
df_R4_Sub = pd.DataFrame(cargar_archivos_a_dataframe(ruta_R4_Sub, Tipo_Archivo, Codificacion_Archivo, Separador_Archivo, tiene_encabezado))
df_S4_Sub = pd.DataFrame(cargar_archivos_a_dataframe(ruta_S4_Sub, Tipo_Archivo, Codificacion_Archivo, Separador_Archivo, tiene_encabezado))
df_R4_Cont = pd.DataFrame(cargar_archivos_a_dataframe(ruta_R4_Cont, Tipo_Archivo, Codificacion_Archivo, Separador_Archivo, tiene_encabezado))
df_S4_Cont = pd.DataFrame(cargar_archivos_a_dataframe(ruta_S4_Cont, Tipo_Archivo, Codificacion_Archivo, Separador_Archivo, tiene_encabezado))
df_S3 = pd.DataFrame(cargar_archivos_a_dataframe(ruta_S3, Tipo_Archivo, Codificacion_Archivo, Separador_Archivo, tiene_encabezado))
df_S5 = pd.DataFrame(cargar_archivos_a_dataframe(ruta_S5, Tipo_Archivo, Codificacion_Archivo, Separador_Archivo, tiene_encabezado))
Codificacion_Archivo = 'UTF-8'

# Define los nombres de las 41 columnas que esperas (puedes personalizarlos si los conoces)
columnas_S6 = [f"Column{i}" for i in range(1, 42)]  # Column1 a Column41

try:
    datos = cargar_archivos_a_dataframe(ruta_S6, Tipo_Archivo, Codificacion_Archivo, Separador_Archivo, tiene_encabezado)
    if datos is not None and len(datos) > 0:
        df_S6 = pd.DataFrame(datos)
        if df_S6.shape[1] != 41:
            print(f"⚠️ Advertencia: Se esperaban 41 columnas pero se encontraron {df_S6.shape[1]}")
    else:
        # Si no hay datos, se crea un DataFrame vacío con las 41 columnas
        df_S6 = pd.DataFrame(columns=columnas_S6)
        print("ℹ️ No se encontraron registros, se creó df_S6 vacío con 41 columnas.")
except Exception as e:
    # Si ocurre algún error, también se crea el DataFrame vacío con 41 columnas
    df_S6 = pd.DataFrame(columns=columnas_S6)
    print("❌ Error al cargar df_S6. Se creó vacío con 41 columnas.")
    print("Detalles del error:", e)


Codificacion_Archivo = 'ANSI'
Tipo_Archivo = '.VAL'
df_S1_Val = pd.DataFrame(cargar_archivos_a_dataframe(ruta_S1_Val, Tipo_Archivo, Codificacion_Archivo, Separador_Archivo, tiene_encabezado))
df_S1_Auto = pd.DataFrame(cargar_archivos_a_dataframe(ruta_S1_Auto, Tipo_Archivo, Codificacion_Archivo, Separador_Archivo, tiene_encabezado))

# Imprimir el número de registros y columnas
num_filas, num_columnas = df_R4_Sub.shape
print("Número de filas df_R4_Sub :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S4_Sub.shape
print("Número de filas df_S4_Sub :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_R4_Cont.shape
print("Número de filas df_R4_Cont :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S4_Cont.shape
print("Número de filas df_S4_Cont :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S1_Val.shape
print("Número de filas df_S1_Val :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S1_Auto.shape
print("Número de filas df_S1_Auto :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S3.shape
print("Número de filas df_S3 :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S5.shape
print("Número de filas df_S5 :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S6.shape
print("Número de filas df_S6 :", num_filas)
print("Número de columnas:", num_columnas)

In [ ]:
def filtrar_por_diccionario(dataframe, filtro_diccionario):
    # Inicializar una máscara booleana con 'True' para todas las filas
    mascara = pd.Series(True, index=dataframe.index)
    
    # Iterar sobre las claves y valores del diccionario de filtro
    for columna, valor in filtro_diccionario.items():
        # Aplicar el filtro para la columna específica
        mascara &= dataframe.iloc[:, columna] == valor
    
    # Aplicar la máscara para filtrar el DataFrame
    dataframe_filtrado = dataframe[mascara]
    
    return dataframe_filtrado

def eliminar_duplicados_por_columnas(df, posiciones_columnas):
    """
    Elimina filas duplicadas en un DataFrame basándose en las columnas especificadas.

    Parámetros:
        df (pandas.DataFrame): El DataFrame del cual eliminar los duplicados.
        posiciones_columnas (list): Una lista de posiciones de columnas a validar duplicados.

    Retorna:
        pandas.DataFrame: El DataFrame resultante después de eliminar los duplicados.
    """
    # Convierte las posiciones de columnas en nombres de columnas
    columnas_a_validar = [df.columns[pos] for pos in posiciones_columnas]
    
    # Elimina los duplicados basados en las columnas especificadas
    df_sin_duplicados = df.drop_duplicates(subset=columnas_a_validar, keep='first')
    
    return df_sin_duplicados

#_______________________________Filtra traslados Aprovados_________________________________
Filtro = {3: "1"}
df_R4_Sub_aprovados = filtrar_por_diccionario(df_R4_Sub, Filtro)
df_S4_Sub_aprovados = filtrar_por_diccionario(df_S4_Sub, Filtro)
df_R4_Cont_aprovados = filtrar_por_diccionario(df_R4_Cont, Filtro)
df_S4_Cont_aprovados = filtrar_por_diccionario(df_S4_Cont, Filtro)

df_R4_Sub_aprovados = eliminar_duplicados_por_columnas(df_R4_Sub_aprovados, [1, 2])
df_S4_Sub_aprovados = eliminar_duplicados_por_columnas(df_S4_Sub_aprovados, [1, 2])
df_R4_Cont_aprovados = eliminar_duplicados_por_columnas(df_R4_Cont_aprovados, [1, 2])
df_S4_Cont_aprovados = eliminar_duplicados_por_columnas(df_S4_Cont_aprovados, [1, 2])


# Para S1_Val
df_S1_Val = df_S1_Val.loc[
    df_S1_Val.iloc[:, 24].isin(["0", "1", "2"]) 
    & ~df_S1_Val.iloc[:, 33].isin(["EPS025", "EPSC25"])
]

# Para S1_Auto
df_S1_Auto = df_S1_Auto.loc[
    df_S1_Auto.iloc[:, 24].isin(["0", "1", "2"]) 
    & ~df_S1_Auto.iloc[:, 33].isin(["EPS025", "EPSC25"])
]

# Para S3
df_S3 = df_S3.loc[
    df_S3.iloc[:, 24].isin(["0", "1", "2", 1, 2, 3]) 
    & ~df_S3.iloc[:, 33].isin(["EPS025", "EPSC25"])
]

# Para S6
df_S6 = df_S6.loc[
    df_S6.iloc[:, 24].isin(["0", "1", "2"]) 
    & ~df_S6.iloc[:, 33].isin(["EPS025", "EPSC25"])
]


# Imprimir el número de registros y columnas
Cat_r4_sub_aprovados , num_columnas = df_R4_Sub_aprovados.shape
print("Número de filas Cat_r4_sub_aprovados  :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
Cat_s4_sub_aprovados , num_columnas = df_S4_Sub_aprovados.shape
print("Número de filas Cat_s4_sub_aprovados  :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
Cat_r4_cont_aprovados , num_columnas = df_R4_Cont_aprovados.shape
print("Número de filas Cat_r4_cont_aprovados  :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
Cat_s4_cont_aprovados , num_columnas = df_S4_Cont_aprovados.shape
print("Número de filas Cat_s4_cont_aprovados  :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S1_Val.shape
print("Número de filas df_S1_Val :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S1_Auto.shape
print("Número de filas df_S1_Auto :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S3.shape
print("Número de filas df_S3 :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S5.shape
print("Número de filas df_S5 :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S6.shape
print("Número de filas df_S6 :", num_filas)
print("Número de columnas:", num_columnas)

In [ ]:
print(df_S5.head())


In [ ]:
df_R4_Sub = eliminar_duplicados_por_columnas(df_R4_Sub, [1, 2, 3])
df_S4_Sub = eliminar_duplicados_por_columnas(df_S4_Sub, [1, 2, 3])
df_R4_Cont = eliminar_duplicados_por_columnas(df_R4_Cont, [1, 2, 3])
df_S4_Cont = eliminar_duplicados_por_columnas(df_S4_Cont, [1, 2, 3])
df_S1_Val = eliminar_duplicados_por_columnas(df_S1_Val, [2, 3])
df_S1_Auto = eliminar_duplicados_por_columnas(df_S1_Auto, [2, 3])
df_S3 = eliminar_duplicados_por_columnas(df_S3, [2, 3])
df_S5 = eliminar_duplicados_por_columnas(df_S5, [1, 8])
df_S5 = df_S5.sort_values(by=df_S5.columns[8], ascending=False)\
             .drop_duplicates(subset=df_S5.columns[[6, 7]], keep='first')
df_S6 = eliminar_duplicados_por_columnas(df_S6, [2, 3])

# Imprimir el número de registros y columnas
num_filas, num_columnas = df_R4_Sub.shape
print("Número de filas df_R4_Sub :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S4_Sub.shape
print("Número de filas df_S4_Sub :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_R4_Cont.shape
print("Número de filas df_R4_Cont :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S4_Cont.shape
print("Número de filas df_S4_Cont :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S1_Val.shape
print("Número de filas df_S1_Val :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S1_Auto.shape
print("Número de filas df_S1_Auto :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S3.shape
print("Número de filas df_S3 :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S5.shape
print("Número de filas df_S5 :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S6.shape
print("Número de filas df_S6 :", num_filas)
print("Número de columnas:", num_columnas)

In [ ]:
def eliminar_registros(df1, cols_df1, df2, cols_df2):
    """
    Elimina los registros de df1 que aparecen en df2 basado en las columnas especificadas.

    Args:
        df1 (pandas.DataFrame): El primer DataFrame.
        cols_df1 (list): La lista de índices de columnas en df1 a considerar para la comparación.
        df2 (pandas.DataFrame): El segundo DataFrame.
        cols_df2 (list): La lista de índices de columnas en df2 a considerar para la comparación.

    Returns:
        pandas.DataFrame: El DataFrame df1 sin los registros que aparecen en df2.
    """
    # Asegurar que las columnas especificadas existen en ambos DataFrames
    if not all(col < len(df1.columns) for col in cols_df1) or not all(col < len(df2.columns) for col in cols_df2):
        raise ValueError("Las columnas especificadas no existen en los DataFrames respectivos.")

    # Eliminar registros de df1 que aparecen en df2 basados en las columnas especificadas
    df1_filtered = df1[~df1.iloc[:, cols_df1].apply(tuple, axis=1).isin(df2.iloc[:, cols_df2].apply(tuple, axis=1))]

    return df1_filtered

df_S5 = eliminar_registros(df_S5, [3, 4], df_S1_Auto, [2, 3])
df_S3 = eliminar_registros(df_S3, [2, 3], df_S5, [3, 4])
df_S3 = eliminar_registros(df_S3, [2, 3], df_S1_Auto, [2, 3])
df_S6 = eliminar_registros(df_S6, [2, 3], df_S5, [3, 4])
df_S6 = eliminar_registros(df_S6, [2, 3], df_S1_Auto, [2, 3])

# Imprimir el número de registros y columnas
num_filas , num_columnas = df_R4_Sub.shape
print("Número de filas df_R4_Sub  :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas , num_columnas = df_S4_Sub.shape
print("Número de filas df_S4_Sub  :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas , num_columnas = df_R4_Cont.shape
print("Número de filas df_R4_Cont  :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas , num_columnas = df_S4_Cont.shape
print("Número de filas df_S4_Cont  :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S1_Val.shape
print("Número de filas df_S1_Val :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S1_Auto.shape
print("Número de filas df_S1_Auto :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S3.shape
print("Número de filas df_S3 :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S5.shape
print("Número de filas df_S5 :", num_filas)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
num_filas, num_columnas = df_S6.shape
print("Número de filas df_S6 :", num_filas)
print("Número de columnas:", num_columnas)

In [ ]:
df_S5_Aprovados = df_S5.loc[df_S5.iloc[:, 8].isin(["1"])]
df_S5_Negados = df_S5.loc[df_S5.iloc[:, 8].isin(["0"])]

num_filas, num_columnas = df_S5_Aprovados.shape
print("Número de filas df_S5_Aprovados :", num_filas)
print("Número de columnas:", num_columnas)
num_filas, num_columnas = df_S5_Negados.shape
print("Número de filas df_S5_Negados :", num_filas)
print("Número de columnas:", num_columnas)

In [ ]:
def eliminar_registros_duplicados_merge(df1, df2, cols):
    
    # Paso 1: Encuentra los índices de las filas en df_1 que coinciden con las filas de df_2 en las columnas [1, 2]
    indices_filas_coincidentes = df1[df1.iloc[:, cols].apply(tuple, axis=1).isin(df2.iloc[:, cols].apply(tuple, axis=1))].index

    # Paso 2: Elimina las filas correspondientes de df_1
    df1 = df1.drop(indices_filas_coincidentes)
    
    # Concatenar los DataFrames filtrados y eliminar duplicados
    df_concatenado = pd.concat([df1, df2]).drop_duplicates()
    
    return df_concatenado


df_R4_Sub = eliminar_registros_duplicados_merge(df_R4_Sub, df_R4_Sub_aprovados, [1, 2])
df_S4_Sub = eliminar_registros_duplicados_merge(df_S4_Sub, df_S4_Sub_aprovados, [1, 2])
df_R4_Cont = eliminar_registros_duplicados_merge(df_R4_Cont, df_R4_Cont_aprovados, [1, 2])
df_S4_Cont = eliminar_registros_duplicados_merge(df_S4_Cont, df_S4_Cont_aprovados, [1, 2])

df_R4_Sub = eliminar_duplicados_por_columnas(df_R4_Sub, [1, 2])
df_S4_Sub = eliminar_duplicados_por_columnas(df_S4_Sub, [1, 2])
df_R4_Cont = eliminar_duplicados_por_columnas(df_R4_Cont, [1, 2])
df_S4_Cont = eliminar_duplicados_por_columnas(df_S4_Cont, [1, 2])

#_______________________________Filtra traslados Negados_________________________________
Filtro = {3: "0"}
df_R4_Sub_Negados = filtrar_por_diccionario(df_R4_Sub, Filtro)
df_S4_Sub_Negados = filtrar_por_diccionario(df_S4_Sub, Filtro)
df_R4_Cont_Negados = filtrar_por_diccionario(df_R4_Cont, Filtro)
df_S4_Cont_Negados = filtrar_por_diccionario(df_S4_Cont, Filtro)

# Imprimir el número de registros y columnas
Cat_r4_sub_negados, num_columnas = df_R4_Sub_Negados.shape
print("Número de filas Cat_r4_sub_negados :", Cat_r4_sub_negados)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
Cat_s4_sub_negados, num_columnas = df_S4_Sub_Negados.shape
print("Número de filas Cat_s4_sub_negados :", Cat_s4_sub_negados)
print("Número de columnas:", num_columnas)

# Imprimir el número de registros y columnas
Cat_r4_cont_negados, num_columnas = df_R4_Cont_Negados.shape
print("Número de filas Cat_r4_cont_negados :", Cat_r4_cont_negados)
print("Número de columnas:", num_columnas)
# Imprimir el número de registros y columnas
Cat_s4_cont_negados, num_columnas = df_S4_Cont_Negados.shape
print("Número de filas Cat_s4_cont_negados :", Cat_s4_cont_negados)
print("Número de columnas:", num_columnas)

In [ ]:
def guardar_excel_con_multiple_hojas(diccionario_dataframes, ruta_excel):
    """
    Guarda múltiples DataFrames en hojas diferentes de un archivo Excel.

    Args:
        diccionario_dataframes (dict): Un diccionario con nombres de hojas como claves y DataFrames como valores.
        ruta_excel (str): Ruta donde se guardará el archivo Excel.

    Returns:
        str: Mensaje indicando que los DataFrames se han guardado en hojas diferentes del archivo Excel.
    """
    # Crea un objeto ExcelWriter
    excel_writer = pd.ExcelWriter(ruta_excel, engine="xlsxwriter")

    # Itera a través del diccionario y guarda cada DataFrame en una hoja
    for nombre_hoja, dataframe in diccionario_dataframes.items():
        dataframe.to_excel(excel_writer, sheet_name=nombre_hoja, index=False)

    # Cierra el objeto ExcelWriter para guardar los cambios
    excel_writer.close()

    return "DataFrames guardados en el archivo Excel en hojas diferentes."

# Crear un diccionario con el nombre de la hoja y el DataFrame correspondiente
diccionario_dataframes = {
    'S4_Sub': df_S4_Sub,
    'R4_Sub': df_R4_Sub,
    'S4_Cont': df_S4_Cont,
    'R4_Cont': df_R4_Cont,
    'df_S1_Val' : df_S1_Val,
    'df_S1_Auto' : df_S1_Auto,
    'df_S3' : df_S3,
    'df_S5' : df_S5,
    'df_S6' : df_S6,
    }

# Llama a la función para guardar los DataFrames en hojas diferentes del archivo Excel
resultado = guardar_excel_con_multiple_hojas(diccionario_dataframes, ruta_excel_salida)
print(resultado)

In [ ]:
wb = load_workbook(filename=ruta_excel_salida)

# Crear una nueva hoja en el libro de trabajo
Resumen = wb.create_sheet("Resumen")

# Obtener las celdas A1 y B1 de la nueva hoja
celda_a1 = Resumen['A1']
celda_a2 = Resumen['A2']
celda_a7 = Resumen['A7']
celda_a8 = Resumen['A8']
celda_a15 = Resumen['A15']
celda_a16 = Resumen['A16']
celda_b2 = Resumen['B2']
celda_b3 = Resumen['B3']
celda_b4 = Resumen['B4']
celda_b5 = Resumen['B5']
celda_b7 = Resumen['B7']
celda_b8 = Resumen['B8']
celda_b9 = Resumen['B9']
celda_b10 = Resumen['B10']
celda_b11 = Resumen['B11']
celda_b16 = Resumen['B16']
celda_b17 = Resumen['B17']
celda_b18 = Resumen['B18']
celda_b19 = Resumen['B19']
celda_b20 = Resumen['B20']
celda_c2 = Resumen['C2']
celda_c3 = Resumen['C3']
celda_c4 = Resumen['C4']
celda_c5 = Resumen['C5']
celda_c7 = Resumen['C7']
celda_c8 = Resumen['C8']
celda_c9 = Resumen['C9']
celda_c10 = Resumen['C10']
celda_c11 = Resumen['C11']
celda_c16 = Resumen['C16']
celda_c17 = Resumen['C17']
celda_c18 = Resumen['C18']
celda_c19 = Resumen['C19']
celda_c20 = Resumen['C20']
celda_d2 = Resumen['D2']
celda_d3 = Resumen['D3']
celda_d4 = Resumen['D4']
celda_d5 = Resumen['D5']
celda_d7 = Resumen['D7']
celda_d8 = Resumen['D8']
celda_d9 = Resumen['D9']
celda_d10 = Resumen['D10']
celda_d11 = Resumen['D11']
celda_d16 = Resumen['D16']
celda_d17 = Resumen['D17']
celda_d18 = Resumen['D18']
celda_d19 = Resumen['D19']
celda_d20 = Resumen['D20']
# Escribir en la celda A1
celda_a1.value = "CONSOLIDADO A OTRA EPS SUBSIDIADO"
celda_a7.value = "CONSOLIDADO A OTRA EPS CONTRIBUTIVO"
celda_a15.value = "CONSOLIDADO A CAPRESOCA EPS  SUB"
celda_a2.value = F"{Fecha} SUB "
celda_a8.value = F"{Fecha} CONT "
celda_a16.value = F"{Fecha}"
celda_b2.value = "SOLICITADOS"
celda_b8.value = celda_b2.value
celda_b3.value = "APROBADOS"
celda_b9.value = celda_b3.value
celda_b4.value = "NEGADOS"
celda_b10.value = celda_b4.value
celda_b5.value = "TOTAL"
celda_b11.value = celda_b5.value
celda_b16.value = celda_b2.value
celda_b17.value = "ACEPTADOS"
celda_b18.value = "GLOSADOS"
celda_b19.value = celda_b4.value
celda_b20.value = "TOTAL"

celda_c2.value = df_R4_Sub.shape[0] + df_S4_Sub.shape[0]
celda_c3.value = Cat_r4_sub_aprovados + Cat_s4_sub_aprovados
celda_c4.value = Cat_r4_sub_negados + Cat_s4_sub_negados
celda_c5.value = celda_c2.value

celda_c8.value = df_R4_Cont.shape[0] + df_S4_Cont.shape[0]
celda_c9.value = Cat_r4_cont_aprovados + Cat_s4_cont_aprovados
celda_c10.value = Cat_r4_cont_negados + Cat_s4_cont_negados
celda_c11.value = celda_c8.value

celda_c16.value = df_S1_Auto.shape[0] + df_S3.shape[0] + df_S5.shape[0] + df_S6.shape[0]
celda_c17.value = df_S1_Auto.shape[0]+ df_S5_Aprovados.shape[0] 
celda_c18.value = df_S3.shape[0] + df_S6.shape[0]
celda_c19.value = df_S5_Negados.shape[0] 
celda_c20.value = celda_c17.value + celda_c18.value + celda_c19.value

R = ((celda_c2.value) / (celda_c2.value))
celda_d2.value = R
R = ((celda_c3.value) / (celda_c2.value))
celda_d3.value = R
R = ((celda_c4.value) / (celda_c2.value))
celda_d4.value = R
celda_d5.value = celda_d2.value

R = ((celda_c8.value) / (celda_c8.value))
celda_d8.value = R
R = ((celda_c9.value) / (celda_c8.value))
celda_d9.value = R
R = ((celda_c10.value) / (celda_c8.value))
celda_d10.value = R
celda_d11.value = celda_d8.value

R = ((celda_c16.value) / (celda_c16.value))
celda_d16.value = R
R = ((celda_c17.value) / (celda_c16.value))
celda_d17.value = R
R = ((celda_c18.value) / (celda_c16.value))
celda_d18.value = R
R = ((celda_c19.value) / (celda_c16.value))
celda_d19.value = R
celda_d20.value = celda_d16.value

# Combinar las celdas A1:D1 en la nueva hoja
Resumen.merge_cells(start_row=1, start_column=1, end_row=1, end_column=4)
# Combinar las celdas A2:A5 en la nueva hoja
Resumen.merge_cells(start_row=2, start_column=1, end_row=5, end_column=1)

# Combinar las celdas A7:D7 en la nueva hoja
Resumen.merge_cells(start_row=7, start_column=1, end_row=7, end_column=4)
# Combinar las celdas A8:A11 en la nueva hoja
Resumen.merge_cells(start_row=8, start_column=1, end_row=11, end_column=1)

# Combinar las celdas A15:D15 en la nueva hoja
Resumen.merge_cells(start_row=15, start_column=1, end_row=15, end_column=4)
# Combinar las celdas A8:A11 en la nueva hoja
Resumen.merge_cells(start_row=16, start_column=1, end_row=20, end_column=1)

# Alinear el texto al centro de la celda combinada
celda_a1.alignment = Alignment(horizontal='center', vertical='center')
celda_a2.alignment = Alignment(horizontal='center', vertical='center')
celda_a7.alignment = Alignment(horizontal='center', vertical='center')
celda_a8.alignment = Alignment(horizontal='center', vertical='center')
celda_a15.alignment = Alignment(horizontal='center', vertical='center')
celda_a16.alignment = Alignment(horizontal='center', vertical='center')
celda_c2.alignment = Alignment(horizontal='center', vertical='center')
celda_c3.alignment = Alignment(horizontal='center', vertical='center')
celda_c4.alignment = Alignment(horizontal='center', vertical='center')
celda_c5.alignment = Alignment(horizontal='center', vertical='center')
celda_c8.alignment = Alignment(horizontal='center', vertical='center')
celda_c9.alignment = Alignment(horizontal='center', vertical='center')
celda_c10.alignment = Alignment(horizontal='center', vertical='center')
celda_c11.alignment = Alignment(horizontal='center', vertical='center')
celda_d2.alignment = Alignment(horizontal='center', vertical='center')
celda_d3.alignment = Alignment(horizontal='center', vertical='center')
celda_d4.alignment = Alignment(horizontal='center', vertical='center')
celda_d5.alignment = Alignment(horizontal='center', vertical='center')
celda_d8.alignment = Alignment(horizontal='center', vertical='center')
celda_d9.alignment = Alignment(horizontal='center', vertical='center')
celda_d10.alignment = Alignment(horizontal='center', vertical='center')
celda_d11.alignment = Alignment(horizontal='center', vertical='center')
# Aplicar formato a la celda combinada
formato_celda_combinada = celda_a1.font = Font(bold=True)  # Negrita
formato_celda_combinada = celda_a2.font = Font(bold=True)  # Negrita
formato_celda_combinada = celda_a7.font = Font(bold=True)  # Negrita
formato_celda_combinada = celda_a8.font = Font(bold=True)  # Negrita
formato_celda_combinada = celda_a15.font = Font(bold=True)  # Negrita
formato_celda_combinada = celda_a16.font = Font(bold=True)  # Negrita
formato_celda_combinada = celda_a1.fill = PatternFill(start_color="BDD6EE", end_color="BDD6EE", fill_type="solid")  # Color de fondo amarillo
formato_celda_combinada = celda_a7.fill = PatternFill(start_color="BDD6EE", end_color="BDD6EE", fill_type="solid")  # Color de fondo amarillo
formato_celda_combinada = celda_a15.fill = PatternFill(start_color="F8CBAD", end_color="F8CBAD", fill_type="solid")  # Color de fondo amarillo
Resumen.column_dimensions['A'].width = 15  # Ancho de la columna en unidades de caracteres
Resumen.column_dimensions['B'].width = 15  # Ancho de la columna en unidades de caracteres
Resumen.column_dimensions['C'].width = 15  # Ancho de la columna en unidades de caracteres
# Establecer el formato de número de la celda D2 como porcentaje
Resumen['D2'].number_format = numbers.FORMAT_PERCENTAGE
Resumen['D3'].number_format = numbers.FORMAT_PERCENTAGE_00
Resumen['D4'].number_format = numbers.FORMAT_PERCENTAGE_00
Resumen['D5'].number_format = numbers.FORMAT_PERCENTAGE
Resumen['D8'].number_format = numbers.FORMAT_PERCENTAGE
Resumen['D9'].number_format = numbers.FORMAT_PERCENTAGE_00
Resumen['D10'].number_format = numbers.FORMAT_PERCENTAGE_00
Resumen['D11'].number_format = numbers.FORMAT_PERCENTAGE
Resumen['D16'].number_format = numbers.FORMAT_PERCENTAGE
Resumen['D17'].number_format = numbers.FORMAT_PERCENTAGE_00
Resumen['D18'].number_format = numbers.FORMAT_PERCENTAGE_00
Resumen['D19'].number_format = numbers.FORMAT_PERCENTAGE_00
Resumen['D20'].number_format = numbers.FORMAT_PERCENTAGE


# Definir los lados del borde
borde = Border(left=Side(style='thin'),
               right=Side(style='thin'),
               top=Side(style='thin'),
               bottom=Side(style='thin'))
# Definir el rango de celdas a las que se les aplicará el borde (A1:D5)
rango_celdas = Resumen['A1:D5']
# Aplicar el borde al rango de celdas
for fila in rango_celdas:
    for celda in fila:
        celda.border = borde
# Definir el rango de celdas a las que se les aplicará el borde (A7:D11)
rango_celdas = Resumen['A7:D11']
# Aplicar el borde al rango de celdas
for fila in rango_celdas:
    for celda in fila:
        celda.border = borde
# Definir el rango de celdas a las que se les aplicará el borde (A15:D20)
rango_celdas = Resumen['A15:D20']
# Aplicar el borde al rango de celdas
for fila in rango_celdas:
    for celda in fila:
        celda.border = borde


# Guardar los cambios en el libro de trabajo
wb.save(ruta_excel_salida)
# Cerrar el libro de trabajo
wb.close()
#exit()